In [1]:
#pip install tflearn


In [2]:
#pip install tensorflow==1.15 --user


In [32]:
import warnings
warnings.filterwarnings("ignore") # to ignore warnings

In [33]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# things we need for Tensorflow
import numpy as np
import tflearn
import tensorflow as tf
import random as rand

In [61]:
clg_data = r"C:\Users\LOKI99\Desktop\python_clg_ project\ChatBot_APP\Chatbot_development_phase\Chatbot_phase_3\college_json_data_file\college_data.json"

In [62]:
# import our chat-bot intents file
import json
with open(clg_data) as json_data:
    intents = json.load(json_data)

In [63]:
intents

{'intents': [{'tag': 'Greetings',
   'patterns': ['hi', 'hey', 'hello', 'namaste'],
   'responses': ['hello', 'hey', 'hi', 'namaste'],
   'context_set': ''},
  {'tag': 'AboutBot',
   'patterns': ['what is your name?',
    'who are you?',
    'what can you do?',
    'what is your work?'],
   'responses': ['Hey, I am loyola bot.I can answer your queries regarding loyola college'],
   'context_set': ''},
  {'tag': 'Built',
   'patterns': ['how are you built?',
    'how do you function?',
    'how can you answer queries?',
    'bot how do you work?',
    'how were you developed?'],
   'responses': ["I'm a chatbot. I'm developed using deep learning & NLP technology"],
   'context_set': ''},
  {'tag': 'Lacollege',
   'patterns': ['what is loyola college?',
    'what is loyola academy degree and pg college?',
    'what is loyola academy?',
    'loyola college?',
    'loyola academy?'],
   'responses': ['Loyola academy degree and pg college is a catholic miniority institution. Loyola college o

In [64]:
words = []
classes = []
documents = []
ignore_words = ['?']
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

235 documents
39 classes ['AboutBot', 'AdmissionProcedure', 'Affiliation', 'ArtsCourses', 'Built', 'BusinessCourses', 'CampusArea', 'CollegeDetails', 'CollegeFacilities', 'CollegeMission', 'CollegeRankings', 'CollegeVision', 'CommerceCourses', 'ContactDetails', 'Correspondent', 'Established', 'FoundationCourses', 'Founder', 'Greetings', 'ItCourses', 'Lacollege', 'Location', 'LoyolaAcademics', 'LoyolaCourses', 'Management', 'PgCourses', 'Principal', 'Rector', 'ScienceCourses', 'TopComp', 'TransFacility', 'UgCourses', 'VicePrincipal', 'academicawards', 'awardshonours', 'fulltimedegree', 'placementdetails', 'researchprogrammes', 'semexams']
127 unique stemmed words ['a', 'about', 'academ', 'academy', 'acedemy', 'act', 'admit', 'affy', 'aft', 'and', 'answ', 'any', 'ar', 'are', 'art', 'at', 'award', 'bot', 'built', 'busy', 'by', 'camp', 'can', 'cert', 'claim', 'colleg', 'commerc', 'company', 'complet', 'conduc', 'contact', 'correspond', 'cours', 'curricul', 'degr', 'detail', 'develop', 'do'

In [65]:
# create our training data
training = []
output = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffle our features and turn into np.array
rand.shuffle(training)
training = np.array(training)

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])

In [66]:
# reset underlying graph data
tf.reset_default_graph()
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save(r'C:\Users\LOKI99\Desktop\python_clg_ project\ChatBot_APP\Chatbot_development_phase\Chatbot_phase_3\college_model_file\softmax_model\college_softmax_model.tflearn')

Training Step: 29999  | total loss: 0.00009 | time: 0.171s
| Adam | epoch: 1000 | loss: 0.00009 - acc: 1.0000 -- iter: 232/235
Training Step: 30000  | total loss: 0.00012 | time: 0.176s
| Adam | epoch: 1000 | loss: 0.00012 - acc: 1.0000 -- iter: 235/235
--
INFO:tensorflow:C:\Users\LOKI99\Desktop\python_clg_ project\ChatBot_APP\Chatbot_development_phase\Chatbot_phase_3\college_model_file\softmax_model\college_softmax_model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [67]:
import os
os.chdir(r"C:\Users\LOKI99\Desktop\python_clg_ project\ChatBot_APP\Chatbot_development_phase\Chatbot_phase_3\pickle_file_1")

In [68]:
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )

In [69]:
# restore all of our data structures
import pickle
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

# import our chat-bot intents file
import json
with open(clg_data) as json_data:
    intents = json.load(json_data)

In [70]:
# load our saved model
model.load(r'C:\Users\LOKI99\Desktop\python_clg_ project\ChatBot_APP\Chatbot_development_phase\Chatbot_phase_3\college_model_file\softmax_model\college_softmax_model.tflearn')

INFO:tensorflow:Restoring parameters from C:\Users\LOKI99\Desktop\python_clg_ project\ChatBot_APP\Chatbot_development_phase\Chatbot_phase_3\college_model_file\softmax_model\college_softmax_model.tflearn


In [71]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [72]:
p = bow("loyola college?", words)
print (p)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [73]:
ERROR_THRESHOLD = 0.25
def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # a random response from the intent
                    return print(rand.choice(i['responses']))

            results.pop(0)

In [74]:
 classify('loyola college?')

[('Lacollege', 0.9988368)]

In [75]:
response('tell me about the college?')

1. loyola college is a catholic minority institution
2. education world ranked loyola college 1st at state level & 11th at national level
3. NAAC re-accredited loyola college in 2019 & awarded an 'A' grade
4. our vast campus which is approx 132 acres, makes us hyderabad's largest private college
5. loyola college has a total of 6 blocks,own weather station & reverse osmosis water purification unit
7. loyola emphasizes as much on practical understanding as on theory, with a total of 28 labs.


In [76]:
response("ug courses")

1.  BSc - Chemical Technology
2.  BSc(Honours) - in  Agriculture Science and Rural Development
3  BSc - Computer Science and Engineering
4  BCom Honors
5.  BSc - Electronics Technology
6.  BSc - Computer Systems & Engineering
7.  BCom - Advertising, Sales Promotion & Sales Management
8.  BSc - Biotechnology, Chemistry & Genetics
9.  BCom - General / Business Studies
10. BA - Mass Communication
11. BSc - Food Technology and Management
12. BA - Psychology, English & Journalism
13. BSc - Maths, Statistics & Computer Science
14. BSc - Multimedia and Animation
15. BCom - Computers
16. BBA - Bachelor of Business Administration
17. BCom - International Accounting & Finance
18. BSc - Computer Data Science & Analytics Engineering
19. BCom(Honors) - in Strategic Finance
20. BCom - Business Process Management
21. BSc - Food Science, Nutrition & Dietetics 


In [77]:
response("principal?")

The principal of the loyola college is Rev Fr Dr Pothireddy Anthony SJ.


In [78]:
response("vice principal?")

1. Rev Fr Dr L Joji Reddy SJ is the vice principal of loyola pg college
 2. Rev Fr Dr Balaswamy Sj is the vice principal of loyola ug college


In [79]:
print(response("college details?"))

1. loyola college is a catholic minority institution
2. education world ranked loyola college 1st at state level & 11th at national level
3. NAAC re-accredited loyola college in 2019 & awarded an 'A' grade
4. our vast campus which is approx 132 acres, makes us hyderabad's largest private college
5. loyola college has a total of 6 blocks,own weather station & reverse osmosis water purification unit
7. loyola emphasizes as much on practical understanding as on theory, with a total of 28 labs.
None


In [80]:
response("how to get admission?")

1. Register and Apply :
Interested candidates can apply offline with required information
2. Entrance Test:
Eligible candidates need to appear for an entrance test conducted by Loyola Academy Degree and PG College
3. Personal Interview:
Shortlisted candidates will be called for a personal interview based on the marks obtained in the entrance test


In [81]:
response("tell me about college placements?")

The following are some of the placement details :
1. 350k pa. Companies gladly pay for our talent pool.
2. 100 companies 
3. 300 jobs in 2018
4. We place students only in the best companies
5. 600 internships


In [82]:
response("tell me about campus placements")

The following are some of the placement details :
1. 350k pa. Companies gladly pay for our talent pool.
2. 100 companies 
3. 300 jobs in 2018
4. We place students only in the best companies
5. 600 internships


In [83]:
response("hello")

hi


In [84]:
response("top companies at campus placements")

some of the top companies are Amazon,DELL,Deloitte,Cognizant,Goldman Sachs,Infosys, and so on 


In [85]:
response("top recruiting companies")

some of the top companies are Amazon,DELL,Deloitte,Cognizant,Goldman Sachs,Infosys, and so on 
